In [1]:
import pandas as pd
from tqdm import tqdm
import time
import requests
from bs4 import BeautifulSoup

In [20]:
def nyt_scraper(x):
    URL = x['article_link']
    print(URL)
    r = requests.get(URL) 
#     soup = BeautifulSoup(r.content, 'html5lib')
    soup = BeautifulSoup(r.content, 'lxml')
    if soup.find('h1', attrs = {'itemprop':'headline'}) is not None:
        x["heading"] = soup.find('h1', attrs = {'itemprop':'headline'}).span.text
    else:
        x["heading"] = soup.find('h1', attrs = {'class':'edye5kn2'}).text
    
    if soup.find('a', attrs = {'class':'css-nuvmzp'}) is not None:
        x["topic"] = soup.find('a', attrs = {'class':'css-nuvmzp'}).text
    else:
        x["topic"] = "Not available"
    try:
        x['front_heading_image'] = soup.find('div', attrs = {'class': 'css-bsn42l'}).img['itemid']
        x['front_image_caption'] = soup.find('figcaption', attrs = {'itemprop': 'caption description'}).span.text
    except:
        pass
    if soup.find('span', attrs = {'class': 'last-byline'}) is not None:
        x['author'] = soup.find('span', attrs = {'class': 'last-byline'}).text
    else:
        x["author"] = "Not available"
    if soup.find('time') is not None:
        x['datetime'] = soup.find('time', attrs = {'class': 'e16638kd0'})['datetime']
    else:
        x["datetime"] = "Not available"
    
    article_text = []
    images_dict = dict()
    article_links = []
    youtube_videos = []
#     print(x['article_link'] + '-' *50)
    rows = soup.find('section', attrs = {'itemprop': 'articleBody'})
    youtube_videos.extend([y['src'] for y in rows.findAll('iframe', attrs={'title':'YouTube Video'})])
    article_text.extend([y.text for y in rows.findAll('p')])
    article_links.extend([y['href'] for y in rows.findAll('a')])
    [images_dict.update({y['src']:"Not available"}) if not y.has_attr('item_id') else (images_dict.update({y['itemid']:"Not available"}) if not y.has_attr('alt') else images_dict.update({y['itemid']:y["alt"]})) for y in rows.findAll('img')]
    x['article'] = ' '.join(article_text)
    x['hyperlinks'] = article_links
    x['images_in_article'] = images_dict
    x['youtube_videos'] = youtube_videos

    return x

In [24]:
# nyt_data = dict()
# nyt_data_scraped = dict()

for year in range(2019, 2020):
    print(year)
    location = './data/nyt-wpo-' + str(year) + '-sample.csv'
    nyt_data[year] = pd.read_csv(location, names=['label', 'article_link'])
    nyt_data[year] = nyt_data[year][nyt_data[year]['article_link'].str.contains("www.nytimes.com")]
    nyt_data[year] = nyt_data[year][~nyt_data[year]['article_link'].str.contains("first-draft")]
    nyt_data[year] = nyt_data[year][~nyt_data[year]['article_link'].str.contains("/watching/")]
#     nyt_data[year] = nyt_data[year][~nyt_data[year]['article_link'].str.contains("www.nytimes.com/2019/04/10/style/modern-love-podcast-anna-chlumsky.html")]
    
    start = time.time()
    nyt_data_scraped[year] = nyt_data[year].apply(lambda x: nyt_scraper(x), axis = 1)
    end = time.time()
    time_taken = end - start

    print("Year: ", year)
    print("No of articles: ", nyt_data[year].shape[0])
    print('Time: ',time_taken)

2019
https://www.nytimes.com/2019/04/10/science/falcon-heavy-launch-spacex.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/science/falcon-heavy-launch-spacex.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/world/americas/pence-venezuela-un-envoy.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/us/politics/william-barr-jeffrey-rosen-hearings.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/world/middleeast/benjamin-netanyahu-israel-election.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/science/black-hole-picture.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/movies/black-hole-movies-streaming.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/style/modern-love-podcast-anna-chlumsky.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/business/dealbook/fake-news-facebook-twitter.html?partner=bloomberg
https://www.nytimes.com/2019/04/10/nyregion/nyc-gps-wireless.html?partner=bloomberg
https://www.nytimes.com/2019/04/

In [29]:
for year in range(2016, 2020):
    location = './data/nyt-wpo-' + str(year) + '-sample-scraped.csv'
    nyt_data_scraped[i].to_csv(location, index = False)

In [ ]:
# URL = nyt_data[year]['article_link'][316]
# print(URL)
# r = requests.get(URL) 
# soup = BeautifulSoup(r.content, 'html5lib')
# [print(x.text) for x in soup.findAll('p')]

In [ ]:

# headings = []
# categories = []
# image_links = []
# image_captions = []
# writers = []
# datetimes = []
# for i in tqdm(nyt_data.iterrows()):
#     URL = i[1][1]
#     r = requests.get(URL) 
#     soup = BeautifulSoup(r.content, 'html5lib')
#     headings.append(soup.find('h1', attrs = {'itemprop':'headline'}).span.text)
#     catgories.append(soup.find('a', attrs = {'class':'css-nuvmzp'}).text)
#     try:
#         image_links.append(soup.find('div', attrs = {'class': 'css-bsn42l'}).img['itemid'])
#         image_captions.append(soup.find('figcaption', attrs = {'itemprop': 'caption description'}).span.text)
#     except:
#         pass
#     writers.append(soup.find('span', attrs = {'class': 'last-byline'}).text)
#     datetimes.append(soup.find('time')['datetime'])